In [1]:
from ipywidgets import interact
from itertools import combinations
from math import log

import numpy as np
from plotly.graph_objs import *
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_splits(total_sales, total_tax, sales):
    sales_and_tax = np.vstack([sales, (np.asarray([5, 12, 18, 28]) / 100)])
    tax_rates_of_unk = sales_and_tax[1, np.isnan(sales)]
    sales_of_unk = total_sales - np.sum(sales[~np.isnan(sales)])
    tax_of_unk = total_tax - np.sum(np.prod(sales_and_tax[:, ~np.isnan(sales)], axis=0))
    alpha = (tax_of_unk - sales_of_unk * tax_rates_of_unk[1]) / (tax_rates_of_unk[0] - tax_rates_of_unk[1])
    beta = (tax_of_unk - sales_of_unk * tax_rates_of_unk[0]) / (tax_rates_of_unk[1] - tax_rates_of_unk[0])
    sales[np.isnan(sales)] = np.asarray([alpha, beta])
    if len(list(filter(lambda x: x, list(sales < 0)))) > 0: 
        raise ValueError("Invalid input values. No solution possible.")
    return sales

In [3]:
def get_valid_solutions(total_sales, total_tax, start, interval):
    valid_solutions = []
    for pos in combinations(list(range(4)), 2):
        X = np.linspace(start, total_sales, (total_sales / interval))
        Y = np.linspace(start, total_sales, (total_sales / interval))
        mesh = np.array(np.meshgrid(X, Y)).T.reshape(-1, 2)
        mesh_with_nan = np.full([mesh.shape[0], 4], np.nan)
        mesh_with_nan[:, pos] = mesh
        for vals in mesh_with_nan:
            try:
                valid_solutions.append(get_splits(total_sales, total_tax, vals))
            except ValueError:
                pass
    return np.asarray(valid_solutions)

In [23]:
def plot_solutions(total_sales, total_tax):
    start = interval = 10 ** (int(log(total_sales, 10) - 1))
    solutions = get_valid_solutions(total_sales, total_tax, start, interval)
    if solutions.size == 0:
        print("Invalid input values. No solution Possible.")
        return
    texts = list(map(lambda x: f"""@5%: {int(x[0])}</br></br>@12%: {int(x[1])}</br>@18%: {int(x[2])}</br>@28%: {int(x[3])}""", solutions))
    data = [
        Scatter3d(
            x=solutions[:,0],
            y=solutions[:, 1],
            z=solutions[:, 2],
            mode='markers',
            marker=dict(
                size=4,
                color=solutions[:, 3],
                colorscale='Viridis',   # choose a colorscale
            ),
            text=texts,
            hoverinfo = 'text'
        )
    ]
    layout = Layout(
        title=f'Possible Solutions for Total Sales {total_sales} and Total Tax {total_tax}',
        scene = dict(
            xaxis=dict(
                title='Sales @5%'
            ),
            yaxis=dict(
                title='Sales @12%'
            ),
            zaxis=dict(
                title='Sales @18%'
            )
        )
    )
    fig = Figure(data=data, layout=layout)
    iplot(fig)

In [25]:
interact(plot_solutions,
         total_sales=(1000, 100000, 10000),
         total_tax=(50, 18000, 500),
         continuous_update=False)

interactive(children=(IntSlider(value=41000, description='total_sales', max=100000, min=1000, step=10000), Int…

<function __main__.plot_solutions(total_sales, total_tax)>